In [1]:
import pandas as pd
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import *
from keras.utils import np_utils
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split

Using Theano backend.
Using cuDNN version 6021 on context None
Mapped name None to device cuda: GeForce GTX 1050 Ti (0000:01:00.0)


# Import Data

In [2]:
def read_data(dataset):
    data = pd.read_csv(dataset)
    data.drop(data.columns[[0]], axis=1, inplace=True)
    data = data.sample(frac=1)
    data = data.values
    n = data.shape[1] - 1
    x = data[:,0:n]
    y = data[:,n]
    y_categorical = np_utils.to_categorical(y)
    return (x, y_categorical)

In [3]:
num_features = 29680

In [12]:
x_train, y_categorical_train = read_data('Featurized/train_featurized_reduced_with_class.csv')
x_test, y_categorical_test = read_data('Featurized/test_featurized_with_class.csv')

In [11]:
x_train.shape

array([[ 0.04067797,  0.00340136,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.03669725,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.04807692,  0.00970874,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.04227783,  0.00431779,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.10526316,  0.02352941,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.00895522,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

# Simple

(805L, 100L)


In [72]:

model = Sequential()
def DenseBlock(num):
    model.add(Dropout(0.7))
    model.add(Dense(num))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
def InputBlock(num, shape):
    model.add(Dense(num, input_shape=shape))
def OutputBlock(shape):
    model.add(Dropout(0.6))
    model.add(Dense(shape))
    model.add(Activation('softmax'))
def ConvBlock(filters):
    model.add(Convolution1D(filters, 3, border_mode='same', activation='relu'))
    model.add(MaxPooling1D(2))
    
InputBlock(128, (num_features, ))
DenseBlock(64)
DenseBlock(32)
OutputBlock(2)

In [73]:
op = Adam(lr=0.00001)
model.compile(optimizer=op, loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
def learn(lr, epochs):
    op.lr = lr
    model.fit(x=x_train, y=y_categorical_train, nb_epoch=epochs, batch_size=2048, validation_data=(x_test, y_categorical_test))

In [77]:
learn(0.00005, 20)

Train on 1034 samples, validate on 186 samples
Epoch 1/20
1034/1034 [==============================] - 0s - loss: 0.1768 - acc: 0.9700 - val_loss: 0.4100 - val_acc: 0.8172
Epoch 2/20
1034/1034 [==============================] - 0s - loss: 0.1700 - acc: 0.9710 - val_loss: 0.4081 - val_acc: 0.8172
Epoch 3/20
1034/1034 [==============================] - 0s - loss: 0.1715 - acc: 0.9671 - val_loss: 0.4062 - val_acc: 0.8172
Epoch 4/20
1034/1034 [==============================] - 0s - loss: 0.1751 - acc: 0.9652 - val_loss: 0.4045 - val_acc: 0.8172
Epoch 5/20
1034/1034 [==============================] - 0s - loss: 0.1674 - acc: 0.9710 - val_loss: 0.4026 - val_acc: 0.8172
Epoch 6/20
1034/1034 [==============================] - 0s - loss: 0.1773 - acc: 0.9691 - val_loss: 0.4010 - val_acc: 0.8172
Epoch 7/20
1034/1034 [==============================] - 1s - loss: 0.1718 - acc: 0.9671 - val_loss: 0.3996 - val_acc: 0.8172
Epoch 8/20
1034/1034 [==============================] - 0s - loss: 0.1647 - ac

In [87]:
model.evaluate(x_test, y_categorical_test)

186/186 [==============================] - 0s     


[0.60578838599625451, 0.75268817012028033]

In [68]:
learn(0.00005, 40)

Train on 1034 samples, validate on 186 samples
Epoch 1/40
1034/1034 [==============================] - 0s - loss: 0.1651 - acc: 0.9613 - val_loss: 0.4090 - val_acc: 0.8656
Epoch 2/40
1034/1034 [==============================] - 1s - loss: 0.1632 - acc: 0.9613 - val_loss: 0.4069 - val_acc: 0.8656
Epoch 3/40
1034/1034 [==============================] - 1s - loss: 0.1694 - acc: 0.9574 - val_loss: 0.4049 - val_acc: 0.8656
Epoch 4/40
1034/1034 [==============================] - 1s - loss: 0.1776 - acc: 0.9478 - val_loss: 0.4027 - val_acc: 0.8656
Epoch 5/40
1034/1034 [==============================] - 1s - loss: 0.1643 - acc: 0.9662 - val_loss: 0.4006 - val_acc: 0.8656
Epoch 6/40
1034/1034 [==============================] - 1s - loss: 0.1654 - acc: 0.9507 - val_loss: 0.3987 - val_acc: 0.8656
Epoch 7/40
1034/1034 [==============================] - 1s - loss: 0.1740 - acc: 0.9516 - val_loss: 0.3967 - val_acc: 0.8656
Epoch 8/40
1034/1034 [==============================] - 1s - loss: 0.1590 - ac

## Notes:

 Highest: Train: 91.81 Validation: 86.17 With Dropout: 0.8 LR: 0.00015, Epoch: 20

 Most Stable: Train: 93.95% Validation: 93.01% With Dropout (including the last layer): 0.8 
 1. LR: 0.00015, Epoch: 30
 2. LR: 0.00005, Epoch: 20

In [78]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_26 (Dense)                 (None, 128)           3799168     dense_input_8[0][0]              
____________________________________________________________________________________________________
dropout_19 (Dropout)             (None, 128)           0           dense_26[0][0]                   
____________________________________________________________________________________________________
dense_27 (Dense)                 (None, 64)            8256        dropout_19[0][0]                 
____________________________________________________________________________________________________
batchnormalization_12 (BatchNorm (None, 64)            256         dense_27[0][0]                   
___________________________________________________________________________________________